# Utilizing SKLearn's `ColumnTransformer`

Up until this point, we've done steps like scaling, imputing, and encoding separately, each as their own piece. But SKLearn has a handy class designed to preprocess groups of columns effectively, streamlining your pre-processing steps!

Enter: [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)

## Set Up

[Insurance Costs data](https://www.kaggle.com/mirichoi0218/insurance) (they got the idea for cleaning up the original open source data from [Machine Learning with R](https://www.packtpub.com/product/machine-learning-with-r-third-edition/9781788295864))

Goal: predict insurance charges

In [ ]:
# Initial imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
df = pd.read_csv('data/insurance.csv')

In [ ]:
# explore the data
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include='O')

In [ ]:
# set our X and y
X = df.drop(columns='charges')
y = df['charges']

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

## Define Our Steps

Observations from our initial data exploration: 
- No null values
- 6 input columns and 1 target column (`charges`)
- `age`, `bmi`, and `children` are numeric
- `sex`, `smoker`, and `region` are objects

What preprocessing steps will we need to take?
- Scale our three numeric inputs
- Encode our three object inputs

## Old Way

We have a train test split, and want to avoid data leakage - if we pre-processed our data before splitting, we'd potenitally leak some information about the test set into our training data!

Let's look at the steps we'd take to preprocess our data without data leakage and without using `ColumnTransformer`, scaling using SKLearn's `MinMaxScaler` and encoding using SKLearn's `OneHotEncoder`:

#### Tackle Our Object Columns First

In [ ]:
# Define a list of our object column names
obj_cols = ['sex', 'smoker', 'region']

# Separate our object columns for both train and test
X_train_obj = X_train[obj_cols]
X_test_obj = X_test[obj_cols]

X_train_obj.head()

In [ ]:
# Instantiate and fit our encoder
ohe = OneHotEncoder(drop='first', sparse=False)
ohe.fit(X_train_obj)

In [ ]:
# Encode our train and test sets
X_train_ohe = ohe.transform(X_train_obj)
X_test_ohe = ohe.transform(X_test_obj)

In [ ]:
# Saving these as dataframes, with appropriate column names and index
X_train_ohe = pd.DataFrame(X_train_ohe, 
                           columns = ohe.get_feature_names(input_features = obj_cols),
                           index=X_train.index)
X_test_ohe = pd.DataFrame(X_test_ohe, 
                          columns = ohe.get_feature_names(input_features = obj_cols),
                          index=X_test.index)

X_train_ohe.head()

#### Now Tackle Our Numeric Columns

In [ ]:
# Define a list of our numeric column names
num_cols = ['age', 'bmi', 'children']

# Separate our numeric columns for both train and test
X_train_num = X_train[num_cols]
X_test_num = X_test[num_cols]

In [ ]:
# Instantiate and fit our scaler
# using minmaxscaler because other cols are now binary
scaler = MinMaxScaler() 
scaler.fit(X_train_num)

In [ ]:
# Scale our train and test sets
X_train_scaled = scaler.transform(X_train_num)
X_test_scaled = scaler.transform(X_test_num)

In [ ]:
# Saving these as dataframes, with appropriate column names and index
X_train_scaled = pd.DataFrame(X_train_scaled, 
                              columns = num_cols,
                              index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, 
                             columns = num_cols,
                             index=X_test.index)

X_train_scaled.head()

#### Put Them Back Together

In [ ]:
# Use concat
X_train_processed = pd.concat([X_train_ohe, X_train_scaled], axis=1)
X_test_processed = pd.concat([X_test_ohe, X_test_scaled], axis=1)

X_train_processed.head()

Whew! In order to do just these two preprocessing steps, we had to separate out our columns, process train and test the same (fitting on train then transforming both train and test of course), then put them back together - and to get all the names out we had to make everything dataframes to concat. That's a lot!

## New Way!

Let's see how `ColumnTransformer` simplifies the process...

In [ ]:
# Need to import ColumnTransformer!
from sklearn.compose import ColumnTransformer

In [ ]:
# We've luckily already defined our list of columns
# We'll need these for our ColumnTransformer
print(f"Object Columns: {obj_cols}")
print(f"Numeric Columns: {num_cols}")

`ColumnTransformer` takes a list of transformers, where each item in the list is a tuple with three parts:
1. Nickname for the step (useful for getting out column names)
2. Preprocessor object
3. List of columns to transform with that preprocessor object

> NOTE! Lists of columns used in `ColumnTransformer` must be mutually exclusive. If you put the same column name in multiple steps in a `ColumnTransformer`, you'll get multiple copies of that column each transformed in a different way.

Let's show it in action!

In [ ]:
# Create an columntransformer object
ct = ColumnTransformer(transformers=[
    ('ohe', OneHotEncoder(drop='first'), obj_cols),
    ('scaler', MinMaxScaler(), num_cols)
])

In [ ]:
# Fit and transform
ct.fit(X_train)
X_train_ct = ct.transform(X_train)
X_test_ct = ct.transform(X_test)

In [ ]:
# Showcase the result - initial output is a numpy array
X_train_ct

If we ever want to turn that resulting numpy array into a dataframe, we'll want to get out the column names after transformation - especially for preprocessing steps like `OneHotEncoder` which changes the number of columns.

We can do that using that handy nickname we set when creating the `ColumnTransformer`! After the `ColumnTransformer` has been fit, we can access the `named_transformers_` attribute to access each step individiually, and treat it just like we would a normal preprocessor object on its own.

In [ ]:
# Accessing the resulting column names from 'ohe'
ohe_col_names = ct.named_transformers_['ohe'].get_feature_names(input_features = obj_cols)
ohe_col_names

In [ ]:
# Now let's turn our X_train_ct into a dataframe to see
# Note that steps are done in the order they're passed
pd.DataFrame(X_train_ct,
             columns = [*ohe_col_names, *num_cols], # Using * to unpack lists
             index = X_train.index).head()

What do you think? Easier or more work to use `ColumnTransformer`? Up to you - so long as you always avoid data leakage in your preprocessing steps!